In [ ]:
from datasets import load_dataset
from random import randrange
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer
import mlflow
import shutil
from os.path import dirname

In [ ]:
def remove_dir(dir_path):
    try:
        shutil.rmtree(dir_path)
        print(f"Folder '{dir_path}' has been deleted.")
    except Exception as e:
        # Ignore errors, you can print a message if needed
        print(f"Folder '{dir_path}' has been deleted.")

#model_name = "Salesforce/codegen-350M-multi"
base_model = "codegen-350M-multi"  #Remember to apply this value in the argument parameter of tokenizer, to use local folder instead of attempting to download from HF
merged_model = "merged_ft_model"
training_output = "training_output"
dataset_name = "Shreyasrp/Text-to-SQL"
split = "train[:10%]"
device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Device to be used is {device_map}")
print(f"--------------------------------------\n")
# Remove the model folders if exist
remove_dir(training_output) 
remove_dir(merged_model)

In [ ]:
# BitsAndBytesConfig config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

peft_config = LoraConfig(
      r=16,
      lora_alpha=32,
      lora_dropout=0.05,
      bias="none",
      task_type="CAUSAL_LM",
)

def prompt_instruction_format(sample):
  return f"""Context:
    {sample['instruction']}

    Result:
    {sample['output']}
    """

In [ ]:
dataset = load_dataset(dataset_name, split=split)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, use_cache = False, device_map=device_map)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('codegen-350M-multi')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = True

In [ ]:
trainingArgs = TrainingArguments(
    output_dir=training_output,
    num_train_epochs=3,
    #per_device_train_batch_size=4,
    auto_find_batch_size=True,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    seed=42
)

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

In [ ]:
print("Start Fine-Tuning")
mlflow.set_experiment("Fine-tune TRL")
trainer.train()
print("Training Done")

# save model
trainer.save_model()
print("Model Saved")

In [ ]:
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    trainingArgs.output_dir,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# Merge LoRA adapter with the base model and save the merged model
lora_merged_model = trained_model.merge_and_unload()
lora_merged_model.save_pretrained(merged_model,safe_serialization=True)
tokenizer.save_pretrained(merged_model)

In [ ]:
# Reset the iPython kernel before proceeding with running the following test.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device_map)

#loaded_models = {}
#mymodel1 = "merged_ft_model"
ft_tokenizer = AutoTokenizer.from_pretrained('merged_ft_model',torch_dtype=torch.bfloat16,device_map=device_map)
ft_model = AutoModelForCausalLM.from_pretrained('merged_ft_model',torch_dtype=torch.bfloat16,device_map=device_map)
#loaded_models[mymodel1] = {"tokenizer": tokenizer1, "model": model1}
#tokenizer1 = loaded_models[mymodel1]["tokenizer"]
#base_model = loaded_models[mymodel1]["model"]

#loaded_models = {}
#mymodel2 = base_model
base_tokenizer = AutoTokenizer.from_pretrained('codegen-350M-multi',torch_dtype=torch.bfloat16,device_map=device_map)
base_model = AutoModelForCausalLM.from_pretrained('codegen-350M-multi',torch_dtype=torch.bfloat16,device_map=device_map)
#loaded_models[mymodel2] = {"tokenizer": tokenizer, "model": model}
#tokenizer2 = loaded_models[mymodel2]["tokenizer"]
#lora_merged_model = loaded_models[mymodel2]["model"]

In [ ]:
mytask="CREATE TABLE trip (bus_stop VARCHAR, duration INTEGER), list all the bus stops from which a trip of duration below 100 started."
prompt = f"""
# Instruction:
Use the context below to produce the result
# context:
{mytask}
# result:
"""

input_id1 = ft_tokenizer.encode(prompt, return_tensors="pt").to(device_map)
input_id2 = base_tokenizer.encode(prompt, return_tensors="pt").to(device_map)
attention_mask1 = torch.ones(input_id1.shape, dtype=torch.long).to(device_map)
attention_mask2 = torch.ones(input_id2.shape, dtype=torch.long).to(device_map)
print(f"--------------------------------------\n")
print(f"Prompt:\n{prompt}\n")
print(f"--------------------------------------\n")

print(f"Fine-tuned Model Result :\n")
output_base = ft_model.generate(input_ids=input_id1, do_sample=True, max_new_tokens=100, top_p=0.9,temperature=0.5,attention_mask=attention_mask1)
print(f"{ft_tokenizer.batch_decode(output_base.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"--------------------------------------\n")

print(f"Base Model Result :\n")
output_ft = base_model.generate(input_ids=input_id2, do_sample=True, max_new_tokens=100, top_p=0.9,temperature=0.5,attention_mask=attention_mask2)
print(f"{base_tokenizer.batch_decode(output_ft.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")